In [1]:
import glob
import pandas as pd
import numpy as np
import ast

In [2]:
annotations=glob.glob('/Users/dgandhi/Desktop/MSBA/OPS 812/data/OPP-115/annotations/*')

In [3]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [4]:
annot=pd.concat(annotations_list, ignore_index=True)

In [5]:
annot.columns=['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [6]:
annot=annot.loc[annot['category_name']!= 'Other']

In [7]:
import ast

In [8]:
annot['dict']=annot.attribute_value_pairs.apply(lambda x: ast.literal_eval(x))

In [9]:
text1=[]
labels=[]
for item in annot['dict']:
    for x,y in item.items():
        if y.get('selectedText')==None:
            pass
        else:
            for key, value in y.items():
                if key=='selectedText':
                    text1.append(value)
                if key=='value':
                    labels.append(x+' '+value)

In [10]:
data=pd.DataFrame(text1,columns=['text'])

In [11]:
data['labels']=labels

In [12]:
data['labels']=data.labels.apply(lambda x: x.lower())

In [13]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)

In [14]:
data.shape

(34459, 3)

In [15]:
classes=set(data['labels'])

In [16]:
data=data.loc[~data['labels'].str.startswith('access')]
data=data.loc[~data['labels'].str.startswith('audience')]
data=data.loc[~data['labels'].str.startswith('choice')]
data=data.loc[~data['labels'].str.startswith('notification')]
data=data.loc[~data['labels'].str.contains('change type')]
data=data.loc[~data['labels'].str.contains('does/does')]
data=data.loc[~data['labels'].str.contains('user choice')]
data=data.loc[~data['labels'].str.contains('user type')]
data=data.loc[~data['labels'].str.contains('do not track')]
data=data.loc[~data['labels'].str.endswith('other')]
data=data.loc[~data['labels'].str.endswith('not-selected')]
data=data.loc[~data['labels'].str.contains('collection mode')]
data=data.loc[data['text']!='']
data=data.loc[data['text']!='null']

In [17]:
data.shape

(20750, 3)

In [18]:
classes=set(data['labels'])
len(classes)

68

In [19]:
class_dict=dict(list (enumerate(classes)))

In [20]:
class_numberdict = {}

for key, value in zip(class_dict.keys(), class_dict.values()):
    # Operations on key/value can also be performed.
    class_numberdict[value] = key

In [21]:
data['classes']=data['labels'].apply(lambda x: class_numberdict[x])
data.head()

,index,text,labels,classes
1,1,register on our website or participate in our ...,action first-party collect on website,40
2,2,"personally-identifiable information, such as",personal information type generic personal inf...,13
5,5,certain additional features that allow you to ...,purpose additional service/feature,18
6,11,Sharing this additional information with us al...,purpose personalization/customization,29
7,17,"In addition, we may use the information you pr...",purpose marketing,31


In [22]:
hi=data.groupby('labels').count().sort_values('classes')

In [23]:
hi[hi.index.str.startswith('retention period')]

,index,text,classes
labels,,,
retention period indefinitely,25,25,25
retention period stated period,30,30,30
retention period limited,77,77,77
retention period unspecified,92,92,92


In [24]:
#take the first label assume correct
NoraData=data.groupby('text').first()

In [25]:
NoraData=NoraData.reset_index()

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [27]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) # You can still specify n-grams here.
X = vectorizer.fit_transform(NoraData['text'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
set(NoraData.labels)

{'action first-party collect from user on other websites',
 'action first-party collect in mobile app',
 'action first-party collect on mobile website',
 'action first-party collect on website',
 'action first-party receive from other parts of company/affiliates',
 'action first-party receive from other service/third-party (named)',
 'action first-party receive from other service/third-party (unnamed)',
 'action first-party track user on other websites',
 'action first-party unspecified',
 'action third party collect on first party website/app',
 'action third party receive/shared with',
 'action third party see',
 'action third party track on first party website/app',
 'action third party unspecified',
 'identifiability aggregated or anonymized',
 'identifiability identifiable',
 'identifiability unspecified',
 'personal information type computer information',
 'personal information type contact',
 'personal information type cookies and tracking elements',
 'personal information type 

In [29]:
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import keras
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [30]:
features = X
targets = np.array(NoraData.classes)
x_train, x_test, y_train, y_test = train_test_split(features, targets.reshape(-1,1), test_size = 0.25, random_state=7)

In [31]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0, verbose=True)
rfc.fit(x_train, np.ravel(y_train))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=True,
                       warm_start=False)

In [32]:
rfc.score(x_train, np.ravel(y_train))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.3s finished


0.9803645576747132

In [33]:
rfc.score(x_test, np.ravel(y_test))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished


0.5613257335866583

In [34]:
NoraData.classes.nunique()

68

In [35]:
bye=NoraData.groupby("labels").count().sort_values('text')

In [36]:
bye[bye.index.str.startswith('third party entity')]

,text,index,classes
labels,,,
third party entity other users,50,50,50
third party entity public,59,59,59
third party entity unspecified,70,70,70
third party entity other part of company/affiliate,203,203,203
third party entity named third party,715,715,715
third party entity unnamed third party,859,859,859


In [37]:
NoraData1=NoraData[NoraData['labels'].isin(bye)]

In [38]:
NoraData1.groupby("labels").count().sort_values('text').shape

(0, 3)

In [39]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) # You can still specify n-grams here.
X = vectorizer.fit_transform(NoraData1['text'])
X.toarray()

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
features = X
targets = np.array(NoraData1.classes)
x_train, x_test, y_train, y_test = train_test_split(features, targets.reshape(-1,1), test_size = 0.25, random_state=7)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0, verbose=True)
rfc.fit(x_train, np.ravel(y_train))

In [ ]:
rfc.score(x_train, np.ravel(y_train))

In [ ]:
rfc.score(x_test, np.ravel(y_test))

In [52]:
lkp = pd.read_csv('/Users/dgandhi/Documents/GitHub/ops812-practicum/class_lookup_tbl.csv')
lkp

,Collect/receive/track from other websites,Collect/receive from mobile/website/internal collection,Information acquisition unspecified,Share/collection method with third party stated,Share/collection method with third party not stated,Personal information type - other,Retention purpose stated,Retention period stated,Retention period unspecified,Security measures access and edit stated,Security measure generic,Third party entity named company,Third party entity - public,Third party entity unspecified
0,action first-party collect from user on other ...,action first-party collect in mobile app,action first-party unspecified,action third party collect on first party webs...,action third party unspecified,personal information type health,retention purpose advertising,retention period stated period,retention period indefinitely,security measure privacy training,retention purpose unspecified,third party entity named third party,third party entity other users,third party entity unspecified
1,action first-party receive from other service/...,action first-party collect on mobile website,NaN,action third party receive/shared with,NaN,personal information type survey data,retention purpose marketing,retention period limited,retention period unspecified,security measure secure user authentication,security measure generic,third party entity other part of company/affil...,third party entity public,third party entity unnamed third party
2,action first-party receive from other service/...,action first-party receive from other parts of...,NaN,action third party see,NaN,personal information type personal identifier,retention purpose analytics/research,NaN,NaN,security measure privacy review/audit,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,action third party track on first party websit...,NaN,personal information type social media data,retention purpose service operation and security,NaN,NaN,security measure secure data storage,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,personal information type demographic,retention purpose legal requirement,NaN,NaN,security measure privacy/security program,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,retention purpose perform service,NaN,NaN,security measure secure data transfer,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,security measure data access limitation,NaN,NaN,NaN,NaN


In [69]:
# labels_dict = lkp.dropna(axis=0)
from math import isnan
ldict = lkp.to_dict(orient='list')
ldict.values()

dict_values([['action first-party collect from user on other websites', 'action first-party receive from other service/third-party (named)', 'action first-party receive from other service/third-party (unnamed)', nan, nan, nan, nan], ['action first-party collect in mobile app', 'action first-party collect on mobile website', 'action first-party receive from other parts of company/affiliates', nan, nan, nan, nan], ['action first-party unspecified', nan, nan, nan, nan, nan, nan], ['action third party collect on first party website/app', 'action third party receive/shared with', 'action third party see', 'action third party track on first party website/app', nan, nan, nan], ['action third party unspecified', nan, nan, nan, nan, nan, nan], ['personal information type health', 'personal information type survey data', 'personal information type personal identifier', 'personal information type social media data', 'personal information type demographic', nan, nan], ['retention purpose advertisi

In [71]:
res={k: [elem for elem in v if elem is not np.nan] for k,v in ldict.items()}
print(res)

{'Collect/receive/track from other websites': ['action first-party collect from user on other websites', 'action first-party receive from other service/third-party (named)', 'action first-party receive from other service/third-party (unnamed)'], 'Collect/receive from mobile/website/internal collection': ['action first-party collect in mobile app', 'action first-party collect on mobile website', 'action first-party receive from other parts of company/affiliates'], 'Information acquisition unspecified': ['action first-party unspecified'], 'Share/collection method with third party stated': ['action third party collect on first party website/app', 'action third party receive/shared with', 'action third party see', 'action third party track on first party website/app'], 'Share/collection method with third party not stated': ['action third party unspecified'], 'Personal information type - other': ['personal information type health', 'personal information type survey data', 'personal informat

In [82]:
list(res.keys())

['Collect/receive/track from other websites',
 'Collect/receive from mobile/website/internal collection',
 'Information acquisition unspecified',
 'Share/collection method with third party stated',
 'Share/collection method with third party not stated',
 'Personal information type - other',
 'Retention purpose stated',
 'Retention period stated',
 'Retention period unspecified',
 'Security measures access and edit stated',
 'Security measure generic',
 'Third party entity named company',
 'Third party entity - public',
 'Third party entity unspecified']

In [93]:
t = pd.DataFrame(res.keys(),res.values())
t

,0
"[action first-party collect from user on other websites, action first-party receive from other service/third-party (named), action first-party receive from other service/third-party (unnamed)]",Collect/receive/track from other websites
"[action first-party collect in mobile app, action first-party collect on mobile website, action first-party receive from other parts of company/affiliates]",Collect/receive from mobile/website/internal c...
[action first-party unspecified],Information acquisition unspecified
"[action third party collect on first party website/app, action third party receive/shared with, action third party see, action third party track on first party website/app]",Share/collection method with third party stated
[action third party unspecified],Share/collection method with third party not s...
"[personal information type health, personal information type survey data, personal information type personal identifier, personal information type social media data, personal information type demographic]",Personal information type - other
"[retention purpose advertising, retention purpose marketing, retention purpose analytics/research, retention purpose service operation and security, retention purpose legal requirement, retention purpose perform service]",Retention purpose stated
"[retention period stated period, retention period limited]",Retention period stated
"[retention period indefinitely, retention period unspecified]",Retention period unspecified
"[security measure privacy training, security measure secure user authentication, security measure privacy review/audit, security measure secure data storage, security measure privacy/security program, security measure secure data transfer, security measure data access limitation]",Security measures access and edit stated


In [113]:
t.reset_index(inplace=True)

In [115]:
t.columns = ['v','k']

In [120]:
#t.v.apply(pd.Series).merge(t, left_index=True, right_index=True).melt(id_vars='k', value_name='blah')
t.v.apply(pd.Series).merge(t, left_index=True, right_index=True)

,0,1,2,3,4,5,6,v,k
0,action first-party collect from user on other ...,action first-party receive from other service/...,action first-party receive from other service/...,NaN,NaN,NaN,NaN,[action first-party collect from user on other...,Collect/receive/track from other websites
1,action first-party collect in mobile app,action first-party collect on mobile website,action first-party receive from other parts of...,NaN,NaN,NaN,NaN,"[action first-party collect in mobile app, act...",Collect/receive from mobile/website/internal c...
2,action first-party unspecified,NaN,NaN,NaN,NaN,NaN,NaN,[action first-party unspecified],Information acquisition unspecified
3,action third party collect on first party webs...,action third party receive/shared with,action third party see,action third party track on first party websit...,NaN,NaN,NaN,[action third party collect on first party web...,Share/collection method with third party stated
4,action third party unspecified,NaN,NaN,NaN,NaN,NaN,NaN,[action third party unspecified],Share/collection method with third party not s...
5,personal information type health,personal information type survey data,personal information type personal identifier,personal information type social media data,personal information type demographic,NaN,NaN,"[personal information type health, personal in...",Personal information type - other
6,retention purpose advertising,retention purpose marketing,retention purpose analytics/research,retention purpose service operation and security,retention purpose legal requirement,retention purpose perform service,NaN,"[retention purpose advertising, retention purp...",Retention purpose stated
7,retention period stated period,retention period limited,NaN,NaN,NaN,NaN,NaN,"[retention period stated period, retention per...",Retention period stated
8,retention period indefinitely,retention period unspecified,NaN,NaN,NaN,NaN,NaN,"[retention period indefinitely, retention peri...",Retention period unspecified
9,security measure privacy training,security measure secure user authentication,security measure privacy review/audit,security measure secure data storage,security measure privacy/security program,security measure secure data transfer,security measure data access limitation,"[security measure privacy training, security m...",Security measures access and edit stated
